In [2]:
import io
import csv
import os.path

import numpy as np
import numexpr as ne
import matplotlib.pyplot as plt

%matplotlib qt5

In [3]:
from posarmctools.ekfnavtools import *
%load_ext autoreload
%autoreload 2

In [4]:
blocksPerFile = 6
samplesPerRamp = 6000
rampsPerFile = 1500
samplesPerFile = 9000000
T_files = samplesPerFile / 10e6

hourOffset = 2

## Load navigation data SbgLogEkfNav

In [5]:
prefix = "/home/pleroy/DATA/PoSAR-MC_PIMA_TEST-3/dataLogger/session_0002/2017_10_12/09h00/"

logEkfEuler = prefix + "sbgLogEkfEuler.dat"
logEkfNav  = prefix + "sbgLogEkfNav.dat"
logUtcData = prefix + "sbgLogUtcData.dat"
logGpsPos = prefix + "sbgLogGpsPos.dat"

In [6]:
# logEkfNav
timeStamps = np.loadtxt( logEkfNav, skiprows = 1, usecols = idx_timeStamp )
Long = np.loadtxt( logEkfNav, skiprows = 1, usecols = idx_Long )
Lat = np.loadtxt( logEkfNav, skiprows = 1, usecols = idx_Lat )
Alt = np.loadtxt( logEkfNav, skiprows = 1, usecols = idx_Alt )
vN, vE, vD = np.loadtxt( logEkfNav, skiprows = 1, usecols = (idx_velNorth, idx_velEast, idx_velDown), unpack=True )

In [7]:
Vel = (vN**2 + vE**2 + vD**2)**0.5

In [8]:
#timeStamp status year month day hour minute second nanoSecond gpsTimeOfWeek
idx_h = 5
idx_m = 6
idx_s = 7
idx_nano = 8
utc = np.loadtxt( logUtcData, skiprows = 1, usecols = (0, idx_h, idx_m, idx_s, idx_nano) )
utc_seconds = (utc[1] + hourOffset) * 3600 + utc[2] * 60 + utc[3] + utc[4] * 1e-9

In [9]:
idx_gpsPos_lat = 3
idx_gpsPos_long = 4
idx_gpsPos_alt = 5
idx_gps_lat = 1
idx_gps_long = 2
idx_gps_alt = 3
gps = np.loadtxt( logGpsPos, skiprows = 1, usecols = (0, idx_gpsPos_lat, idx_gpsPos_long, idx_gpsPos_alt) )

In [11]:
printUtc( 0, utc )
printUtc( -1, utc )

8.0 : 59.0 : 59.995
9.0 : 22.0 : 46.230


In [12]:
utc.shape

(273248, 5)

### Plot navigation data jointly with track references

In [13]:
plt.figure()
ax = plt.subplot(111)
plotLongLatAndTrackReferences( ax, Long, Lat )
plt.title( "Navigation data and track references\n" + prefix )
plt.grid()

In [13]:
validNav = np.where(Long < 0)

In [19]:
plt.figure()
ax = plt.subplot(111)
plotLongLatAndTrackReferences( ax, Long[validNav], Lat[validNav] )
plt.plot( gps[:,idx_gps_long], gps[:,idx_gps_lat], 'lime' )
plt.title( "Navigation data and track references\n" + prefix )
plt.grid()

### Check the existence of record files

In [ ]:
firstRecord = 0
lastRecord = 390
aux = np.zeros(3372)
fileList = []
data_dir = "/home/pleroy/DATA/PoSAR-MC_PIMA_TEST-3/2017_10_10_13_52_12"
for k in range(firstRecord, lastRecord, blocksPerFile):
    filename = data_dir + "/record" + str(k) + ".bin"
    ret = os.path.isfile(filename)
    if ret == False:
        fileList.append(0)
    else:
        fileList.append(1)
idxFileList = np.arange( len( fileList ) ) * blocksPerFile + firstRecord

In [ ]:
plt.plot(idxFileList, fileList, 'o-')

In [ ]:
# times recorded by Windows XP
t_0    = (13, 52, 12)

record0Start = firstRecord
record0Stop = lastRecord

Look for the time data corresponding to the records

In [ ]:
# compute time of day using the date of block 0 and a computed delta
record0_tStart = getTimeOfDay( getSeconds( t_0 ) + record0Start / blocksPerFile * T_files )
record0_tStop = getTimeOfDay( getSeconds( t_0 ) + record0Stop / blocksPerFile * T_files )

In [ ]:
record0_idx = getIndices (record0_tStart, record0_tStop, utc_seconds)

### Process timeStamps.data

In [ ]:
data_dir = "/home/pleroy/DATA/PoSAR-MC_PIMA_TEST-3/2017_10_10_13_37_22"
timeStampsFile = data_dir + "/2017_10_10_13_37_22_timeStamps.data"
blockNumber_1, timeStamp_1 = np.loadtxt( timeStampsFile, skiprows = 1, unpack=True )

In [ ]:
data_dir = "/home/pleroy/DATA/PoSAR-MC_PIMA_TEST-3/2017_10_10_13_45_04"
timeStampsFile = data_dir + "/2017_10_10_13_45_04_timeStamps.data"
blockNumber_2, timeStamp_2 = np.loadtxt( timeStampsFile, skiprows = 1, unpack=True )

In [ ]:
data_dir = "/home/pleroy/DATA/PoSAR-MC_PIMA_TEST-3/2017_10_10_13_52_12"
timeStampsFile = data_dir + "/2017_10_10_13_52_12_timeStamps.data"
blockNumber_3, timeStamp_3 = np.loadtxt( timeStampsFile, skiprows = 1, unpack=True )

In [ ]:
timeStampStart_1 = timeStamp_1[0]
timeStampStop_1 = timeStamp_1[-1]

timeStampStart_2 = timeStamp_2[0]
timeStampStop_2 = timeStamp_2[-1]

timeStampStart_3 = timeStamp_3[0]
timeStampStop_3 = timeStamp_3[-1]

In [ ]:
idxStart = np.where(utc_timeStamp <= timeStampStart_1)[0][-1]
idxStop = np.where(utc_timeStamp <= timeStampStop_1)[0][-1]
record1_idx_alt = (idxStart, idxStop)

idxStart = np.where(utc_timeStamp <= timeStampStart_2)[0][-1]
idxStop = np.where(utc_timeStamp <= timeStampStop_2)[0][-1]
record2_idx_alt = (idxStart, idxStop)

idxStart = np.where(utc_timeStamp <= timeStampStart_3)[0][-1]
idxStop = np.where(utc_timeStamp <= timeStampStop_3)[0][-1]
record3_idx_alt = (idxStart, idxStop)

### Plot navigation data and record periods

In [ ]:
plt.figure()

ax = plt.subplot(111)

plotLongLatAndTrackReferences( ax, Long[validNav], Lat[validNav] )
plotRunaway( ax )

#addOnPlot( ax, Long, Lat, record0_idx, 'yellow', 'record 13_37_22 (time line dated by the MEB)' )
addOnPlot( ax, Long, Lat, record1_idx_alt, 'orange', 'record 13_37_22 (time line dated by SBG time stamps)' )
addOnPlot( ax, Long, Lat, record2_idx_alt, 'red', 'record 13_45_04 (time line dated by SBG time stamps)' )
addOnPlot( ax, Long, Lat, record3_idx_alt, 'green', 'record 13_52_12 (time line dated by SBG time stamps)' )

plt.title("Navigation data, track references, record locations\nFlight 2017_10_10_11h00")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.legend()
plt.grid()

## Plot attitude data

In [ ]:
idx_timeStamp = 0
idx_roll = 1
idx_pitch = 2
idx_yaw = 3
idx_rollStdDev = 4
idx_pitchStdDev = 5
idx_yawStdDev = 6
idx_status = 7

In [ ]:
euler_timeStamp, roll, pitch, yaw = np.loadtxt( logEkfEuler, skiprows = 1, 
                                               usecols = (0, idx_roll, idx_pitch, idx_yaw), unpack=True )

### Track 1

In [ ]:
idxRange = range(record1_idx_alt[0], record1_idx_alt[1])

plt.figure()

plt.subplot(211)
plt.title("Altitude and velocity during record record 13_37_22")
plt.plot( Alt[idxRange], label="altitude" )
plt.legend()
plt.grid()
plt.subplot(212)
plt.plot( Vel[idxRange], label="velocity" )
plt.legend()
plt.grid()

In [ ]:
plt.figure()

plt.subplot(311)
plt.title("Pitch roll and yaw during record record 13_37_22")
plt.plot( pitch[idxRange], label="pitch" )
plt.legend()
plt.grid()

plt.subplot(312)
plt.plot( roll[idxRange], label="roll" )
plt.legend()
plt.grid()

plt.subplot(313)
plt.plot( yaw[idxRange], label="yaw" )
plt.legend()
plt.grid()